In [ ]:
# %% Deep learning - Section 11.113
#    Scrambled MNIST

# This code pertains a deep learning course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/deeplearning_x
# The "base" code in this repository is adapted (with very minor modifications)
# from code developed by the course instructor (Mike X. Cohen), while the
# "exercises" and the "code challenges" contain more original solutions and
# creative input from my side. If you are interested in DL (and if you are
# reading this statement, chances are that you are), go check out the course, it
# is singularly good.


In [ ]:
# %% Libraries and modules
import numpy               as np
import matplotlib.pyplot   as plt
import torch
import torch.nn            as nn
import seaborn             as sns
import copy
import torch.nn.functional as F
import pandas              as pd
import scipy.stats         as stats
import time

from torch.utils.data                 import DataLoader,TensorDataset
from sklearn.model_selection          import train_test_split
from google.colab                     import files
from torchsummary                     import summary
from IPython                          import display
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')


In [ ]:
# %% Data

# Load data
data = np.loadtxt(open('sample_data/mnist_train_small.csv','rb'),delimiter=',')

# Split labels from data
labels = data[:,0]
data   = data[:,1:]

# Normalise data (original range is (0,255))
data_norm = data / np.max(data)


In [ ]:
# %% Scrambled data

permute   = np.random.permutation(data.shape[1])
scrambled = data_norm[:,permute]

# show a few random digits
phi = ( 1 + np.sqrt(5) ) / 2
fig,axs = plt.subplots(3,4,figsize=(1.5*6*phi,6))

for ax in axs.flatten():

  randimg2show = np.random.randint(0,high=data.shape[0])

  img = np.reshape(scrambled[randimg2show,:],(28,28))
  ax.imshow(img,cmap='gray')
  ax.set_title('The number %i'%labels[randimg2show])

plt.suptitle('The scrambled data',fontsize=14)
plt.tight_layout(rect=[0,0,1,.95])

plt.savefig('figure57_scrambled_mnist.png')

plt.show()

files.download('figure57_scrambled_mnist.png')


In [ ]:
# %% Create train and test datasets

# Convert to tensor (float and integers) the scrambled data
data_tensor   = torch.tensor(scrambled).float()
labels_tensor = torch.tensor(labels).long()

# Split data with scikitlearn (10% test data)
train_data,test_data,train_labels,test_labels = train_test_split(data_tensor,labels_tensor,test_size=0.1)

# Convert to PyTorch datasets
train_data = TensorDataset(train_data,train_labels)
test_data  = TensorDataset(test_data,test_labels)

# Convert into DataLoader objects
batch_size   = 32
train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,drop_last=True)
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])


In [ ]:
# %% Function to generate the model

def gen_model():

    class mnist_FFN(nn.Module):
        def __init__(self):
            super().__init__()

            # Architecture
            self.input  = nn.Linear(784,64)
            self.fc1    = nn.Linear( 64,32)
            self.fc2    = nn.Linear( 32,32)
            self.output = nn.Linear( 32,10)

        # Forward propagation
        def forward(self,x):

            x = F.relu(self.input(x))
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.output(x)

            return x

    # Create model instance
    ANN = mnist_FFN()

    # Loss function
    loss_fun = nn.CrossEntropyLoss()

    # Optimizer (SGD to slow down learning for illustration purpose)
    optimizer = torch.optim.SGD(ANN.parameters(),lr=0.01)

    return ANN,loss_fun,optimizer


In [ ]:
# %% Function to train the model

def train_model():

    # Parameters, model instance, inizialise vars
    num_epochs = 60
    ANN,loss_fun,optimizer = gen_model()

    losses_trn = []
    losses_tst = []
    train_acc  = []
    test_acc   = []

    # Loop over epochs
    for epoch_i in range(num_epochs):

        # Loop over training batches
        batch_acc  = []
        batch_loss = []

        for X,y in train_loader:

            # Forward propagation and loss
            yHat = ANN(X)
            loss = loss_fun(yHat,y)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Loss and accuracy from this batch
            batch_loss.append(loss.item())

            matches     = torch.argmax(yHat,axis=1) == y
            matches_num = matches.float()
            accuracy    = 100 * torch.mean(matches_num)
            batch_acc.append(accuracy)

        losses_trn.append( np.mean(batch_loss) )
        train_acc.append( np.mean(batch_acc) )

        # Test accuracy
        ANN.eval()

        with torch.no_grad():
            X,y = next(iter(test_loader))
            yHat = ANN(X)
        test_acc.append( 100*torch.mean((torch.argmax(yHat,axis=1)==y).float()) )
        loss = loss_fun(yHat,y)
        losses_tst.append(loss.item())

        ANN.train()

    return train_acc,test_acc,losses_trn,losses_tst,ANN


In [ ]:
# %% Fit the model

train_acc,test_acc,losses_trn,losses_tst,ANN = train_model()


In [ ]:
# %% Plotting

phi = ( 1 + np.sqrt(5) ) / 2
fig,ax = plt.subplots(1,2,figsize=(1.5*6*phi,6))

ax[0].plot(losses_trn,label='Train loss')
ax[0].plot(losses_tst,label='Test loss')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss')
ax[0].set_ylim([0,3])
ax[0].set_title('Model loss')
ax[0].legend()

ax[1].plot(train_acc,label='Train accuracy')
ax[1].plot(test_acc,label='Test accuracy')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Accuracy (%)')
ax[1].set_ylim([10,100])
ax[1].set_title(f'Final model test accuracy: {test_acc[-1]:.2f}%\nNormalised train and test data')
ax[1].legend()

plt.savefig('figure58_scrambled_mnist.png')

plt.show()

files.download('figure58_scrambled_mnist.png')


In [ ]:
# %% Exercise 1
#    We scrambled all images using the same permuted index. What would happen if each image is uniquely
#    randomly scrambled? Rewrite the scrambling code so that each individual stimulus has a unique,
#    random permuted index. What do the results show and why is that the case?

# So, in this case the performace collapses. It makes sense because the model
# can learn patterns, even though not spatial patterns in this case, and the
# shuffling with one index basically preserves the statistical structure of the
# digits, despite looking unrecognaisable to an human eye. This is already
# interesting beacuse it shows how these models do not "recognise" digits, they
# simply classify images based on their statistics, assigning them to different
# classes also based on the nuber of units in the output layers (i.e., the model
# also works because we know in advance we are classifying 10 numbers); and this
# is indeed possibe even when the images are shuffled, but consistently. If,
# however, the images are "inconsistently" scrambled, the model cannot learn to
# classify the images, because there is no statistical structure to build upon.

# Unique scambling (preallocate and run)
scrambled = np.zeros_like(data_norm)

for i in range(data_norm.shape[0]):
    permute      = np.random.permutation(data_norm.shape[1])
    scrambled[i] = data_norm[i,permute]
